##### Copyright 2021 The TensorFlow Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# TFX Pipeline 및  TensorFlow Data Validation을 사용한 데이터 검증

참고: 설정이 필요하지 않은 Colab 노트북에서 이 튜토리얼을 실행하는 것이 좋습니다! "Google Colab에서 실행하기"를 클릭하기만 하면 됩니다.

<div class="devsite-table-wrapper"><table class="tfo-notebook-buttons" align="left">
<td><a target="_blank" href="https://www.tensorflow.org/tfx/tutorials/tfx/penguin_tfdv"> <img src="https://www.tensorflow.org/images/tf_logo_32px.png">TensorFlow.org에서 보기</a></td>
<td><a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs-l10n/blob/master/site/ko/tfx/tutorials/tfx/penguin_tfdv.ipynb"> <img src="https://www.tensorflow.org/images/colab_logo_32px.png">Google Colab에서 실행하기</a></td>
<td><a target="_blank" href="https://github.com/tensorflow/docs-l10n/blob/master/site/ko/tfx/tutorials/tfx/penguin_tfdv.ipynb"> <img width="32px" src="https://www.tensorflow.org/images/GitHub-Mark-32px.png">GitHub에서 소스 보기</a></td>
<td><a href="https://storage.googleapis.com/tensorflow_docs/docs-l10n/site/ko/tfx/tutorials/tfx/penguin_tfdv.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png">노트북 다운로드</a></td>
</table></div>

이 노트북 기반 튜토리얼에서는, TFX 파이프라인을 만들고 실행하여 입력 데이터를 검증하고 ML 모델을 생성합니다. 이 노트북은 [단순 TFX 파이프라인 튜토리얼](https://www.tensorflow.org/tfx/tutorials/tfx/penguin_simple)에서 구축한 TFX 파이프라인을 기반으로 합니다. 아직 해당 튜토리얼을 읽지 않았다면, 이 노트북을 진행하기 전에 읽어보길 바랍니다.

데이터 과학 또는 ML 프로젝트의 첫 번째 작업은 다음과 같이 데이터를 이해하고 정리하는 것입니다.

- 각 특성에 대한 데이터 유형, 분포 및 기타 정보(예: 평균 값 또는 고유 수) 이해
- 데이터를 설명하는 예비 스키마 생성
- 특정 스키마에 관련된 데이터 내의 이상 및 누락된 값 식별

이 튜토리얼에서는, 두 가지 TFX 파이프라인을 생성할 것입니다.

우선 하나의 파이프라인을 생성하여 데이터세트를 분석하고 주어진 데이터시트의 예비 스키마를 생성합니다. 이 파이프라인은 두 가지 새로운 구성 요소인 `StatisticsGen` 및 `SchemaGen`를 포함합니다.

데이터의 적절한 스키마가 생성되면, 파이프라인을 생성하여 이전 튜토리얼의 파이프라인을 바탕으로 한 ML 분류 모델을 훈련합니다. 이 튜토리얼에서는, 첫 번째 파이프라인의 스키마와 새로운 구성 요소인 `ExampleValidator`를 사용하여 입력 데이터를 검증할 것입니다.

세 가지 새로운 구성 요소인 StatisticsGen, SchemaGen 및 ExampleValidator는 데이터 분류 및 검증을 위한 TFX 구성 요소로, [TensorFlow Data Validation](https://www.tensorflow.org/tfx/guide/tfdv) 라이브러리를 사용하여 구현됩니다.

TFX의 다양한 개념에 대해 자세히 알아보려면 [TFX 파이프라인 이해하기](https://www.tensorflow.org/tfx/guide/understanding_tfx_pipelines)를 참조하세요.

## 설정하기

먼저 TFX Python 패키지를 설치하고 모델에 사용할 데이터세트를 다운로드해야 합니다.

### Pip 업그레이드

로컬에서 실행할 때 시스템에서 Pip을 업그레이드하지 않으려면 Colab에서 실행 중인지 확인해야 합니다. 물론 로컬 시스템은 별도로 업그레이드할 수 있습니다.

In [ ]:
try:
  import colab
  !pip install --upgrade pip
except:
  pass

### TFX 설치


In [ ]:
!pip install -U tfx

### shapely 설치 제거하기

TODO(b/263441833) ImportError를 피하는 임시 솔루션입니다. 다른 추가 종속성을 제거하는 대신 최신 버전의 Bigquery를 지원하여 처리하는 것이 이상적입니다.

In [ ]:
!pip uninstall shapely -y

### 런타임을 다시 시작했습니까?

Google Colab을 사용하는 경우, 위의 셀을 처음 실행한다면 위의 "런타임 다시 시작" 버튼을 클릭하거나 "런타임 &gt; 런타임 다시 시작 ..." 메뉴을 사용하여 런타임을 다시 시작해야 합니다. 이는 Colab이 패키지를 로드하는 방식 때문입니다.

TensorFlow 및 TFX 버전을 확인합니다.

In [ ]:
import tensorflow as tf
print('TensorFlow version: {}'.format(tf.__version__))
from tfx import v1 as tfx
print('TFX version: {}'.format(tfx.__version__))

### 변수 설정하기

파이프라인을 정의하는 데 사용되는 변수가 몇 가지 있습니다. 이러한 변수를 원하는 대로 사용자 정의할 수 있습니다. 기본적으로 파이프라인의 모든 출력은 현재 디렉터리 아래에 생성됩니다.

In [ ]:
import os

# We will create two pipelines. One for schema generation and one for training.
SCHEMA_PIPELINE_NAME = "penguin-tfdv-schema"
PIPELINE_NAME = "penguin-tfdv"

# Output directory to store artifacts generated from the pipeline.
SCHEMA_PIPELINE_ROOT = os.path.join('pipelines', SCHEMA_PIPELINE_NAME)
PIPELINE_ROOT = os.path.join('pipelines', PIPELINE_NAME)
# Path to a SQLite DB file to use as an MLMD storage.
SCHEMA_METADATA_PATH = os.path.join('metadata', SCHEMA_PIPELINE_NAME,
                                    'metadata.db')
METADATA_PATH = os.path.join('metadata', PIPELINE_NAME, 'metadata.db')

# Output directory where created models from the pipeline will be exported.
SERVING_MODEL_DIR = os.path.join('serving_model', PIPELINE_NAME)

from absl import logging
logging.set_verbosity(logging.INFO)  # Set default logging level.

### 예제 데이터 준비하기

TFX 파이프라인에서 사용할 예제 데이터세트를 다운로드합니다. 사용하는 데이터세트는 다른 [TFX 예제](https://allisonhorst.github.io/palmerpenguins/articles/intro.html)에서도 사용하는 [Palmer Penguins 데이터세트](https://github.com/tensorflow/tfx/tree/master/tfx/examples/penguin)입니다.

이 데이터세트에는 4가지 숫자 특성이 있습니다.

- culmen_length_mm
- culmen_depth_mm
- flipper_length_mm
- body_mass_g

모든 특성이 이미 [0,1] 범위를 갖도록 정규화되었습니다. 펭귄의 `species`를 예측하는 분류 모델을 빌드합니다.

TFX ExampleGen 구성 요소는 디렉터리로부터 입력을 읽기 때문에 디렉터리를 생성한 후 데이터세트를 디렉터리에 복사해야 합니다.

In [ ]:
import urllib.request
import tempfile

DATA_ROOT = tempfile.mkdtemp(prefix='tfx-data')  # Create a temporary directory.
_data_url = 'https://raw.githubusercontent.com/tensorflow/tfx/master/tfx/examples/penguin/data/labelled/penguins_processed.csv'
_data_filepath = os.path.join(DATA_ROOT, "data.csv")
urllib.request.urlretrieve(_data_url, _data_filepath)

CSV 파일을 간단히 살펴보세요.

In [ ]:
!head {_data_filepath}

다섯 가지 특성 열을 볼 수 있어야 합니다. `species`은 0, 1 또는 2 중 하나이며 모든 기타 특성은 0과 1 사이의 값을 가져야 합니다. TFX 파이프라인을 생성하여 이 데이터세트를 분석하겠습니다.

## 예비 스키마 생성

TFX 파이프라인은 Python API을 사용하여 정의됩니다. 파이프라인을 생성하여 입력 예제에서 스키마를 자동으로 생성하겠습니다. 이 스키마는 인간이 검토할 수 있으며 필요한 대로 조정할 수 있습니다. 스키마가 완료되면 다음 작업에서 훈련 및 예제 검증에 사용할 수 있습니다.

[단순 TFX 파이프라인 튜토리얼](https://www.tensorflow.org/tfx/tutorials/tfx/penguin_simple)에서 사용되는 `CsvExampleGen`에 추가로, `StatisticsGen` 및 `SchemaGen`을 사용하겠습니다.

- [StatisticsGen](https://www.tensorflow.org/tfx/guide/statsgen)은 데이터세트에 대한 통계를 계산합니다.
- [SchemaGen](https://www.tensorflow.org/tfx/guide/schemagen)은 통계를 검사하고 초기 데이터 스키마를 생성합니다.

각 구성 요소 또는 [TFX 구성 요소 튜토리얼](https://www.tensorflow.org/tfx/tutorials/tfx/components_keras)에 대한 가이드를 확인하여 이러한 구성 요소에 대해 더 자세히 학습합니다.

### 파이프라인 정의 작성하기

TFX 파이프라인을 생성하는 함수를 정의합니다. `Pipeline` 객체는 TFX가 지원하는 파이프라인 오케스트레이션 시스템 중 하나를 사용하여 실행할 수 있는 TFX 파이프라인을 나타냅니다.

In [ ]:
def _create_schema_pipeline(pipeline_name: str,
                            pipeline_root: str,
                            data_root: str,
                            metadata_path: str) -> tfx.dsl.Pipeline:
  """Creates a pipeline for schema generation."""
  # Brings data into the pipeline.
  example_gen = tfx.components.CsvExampleGen(input_base=data_root)

  # NEW: Computes statistics over data for visualization and schema generation.
  statistics_gen = tfx.components.StatisticsGen(
      examples=example_gen.outputs['examples'])

  # NEW: Generates schema based on the generated statistics.
  schema_gen = tfx.components.SchemaGen(
      statistics=statistics_gen.outputs['statistics'], infer_feature_shape=True)

  components = [
      example_gen,
      statistics_gen,
      schema_gen,
  ]

  return tfx.dsl.Pipeline(
      pipeline_name=pipeline_name,
      pipeline_root=pipeline_root,
      metadata_connection_config=tfx.orchestration.metadata
      .sqlite_metadata_connection_config(metadata_path),
      components=components)

### 파이프라인 실행

이전 튜토리얼과 같이 `LocalDagRunner`를 사용합니다.

In [ ]:
tfx.orchestration.LocalDagRunner().run(
  _create_schema_pipeline(
      pipeline_name=SCHEMA_PIPELINE_NAME,
      pipeline_root=SCHEMA_PIPELINE_ROOT,
      data_root=DATA_ROOT,
      metadata_path=SCHEMA_METADATA_PATH))

파이프라인이 성공적으로 완료되면 "INFO:absl:Component SchemaGen is finished." 메시지가 표시됩니다.

파이프라인의 출력을 검사하여 데이터세트에 대해 이해합니다.

### 파이프라인의 출력 검토

이전 튜토리얼에서 설명한 바와 같이 TFX 파이프라인은 두 가지 종류의 출력인, 아티팩트 및 파이프라인 실행의 메타데이터를 포함하는 아티팩트 및 [metadata DB(MLMD)](https://www.tensorflow.org/tfx/guide/mlmd)를 생성합니다. 출력 위치는 위의 셀에 정의되어 있습니다. 기본적으로, 아티팩트는 `pipelines` 디렉터리 아래 저장되며 메타데이터는 `metadata` 디렉터리 아래 sqlite 데이터베이스로 저장됩니다.

이러한 출력을 MLMD API를 사용하여 프로그래밍 방식으로 찾을 수 있습니다. 우선, 일부 유틸리티 함수를 정의하여 방금 생성한 출력 아티팩트를 찾을 수 있습니다.


In [ ]:
from ml_metadata.proto import metadata_store_pb2
# Non-public APIs, just for showcase.
from tfx.orchestration.portable.mlmd import execution_lib

# TODO(b/171447278): Move these functions into the TFX library.

def get_latest_artifacts(metadata, pipeline_name, component_id):
  """Output artifacts of the latest run of the component."""
  context = metadata.store.get_context_by_type_and_name(
      'node', f'{pipeline_name}.{component_id}')
  executions = metadata.store.get_executions_by_context(context.id)
  latest_execution = max(executions,
                         key=lambda e:e.last_update_time_since_epoch)
  return execution_lib.get_output_artifacts(metadata, latest_execution.id)

# Non-public APIs, just for showcase.
from tfx.orchestration.experimental.interactive import visualizations

def visualize_artifacts(artifacts):
  """Visualizes artifacts using standard visualization modules."""
  for artifact in artifacts:
    visualization = visualizations.get_registry().get_visualization(
        artifact.type_name)
    if visualization:
      visualization.display(artifact)

from tfx.orchestration.experimental.interactive import standard_visualizations
standard_visualizations.register_standard_visualizations()

이제 파이프라인 실행의 출력을 검사할 수 있습니다.

In [ ]:
# Non-public APIs, just for showcase.
from tfx.orchestration.metadata import Metadata
from tfx.types import standard_component_specs

metadata_connection_config = tfx.orchestration.metadata.sqlite_metadata_connection_config(
    SCHEMA_METADATA_PATH)

with Metadata(metadata_connection_config) as metadata_handler:
  # Find output artifacts from MLMD.
  stat_gen_output = get_latest_artifacts(metadata_handler, SCHEMA_PIPELINE_NAME,
                                         'StatisticsGen')
  stats_artifacts = stat_gen_output[standard_component_specs.STATISTICS_KEY]

  schema_gen_output = get_latest_artifacts(metadata_handler,
                                           SCHEMA_PIPELINE_NAME, 'SchemaGen')
  schema_artifacts = schema_gen_output[standard_component_specs.SCHEMA_KEY]

이제 각 구성 요소의 출력을 검사할 때입니다. 위에 설명된 대로, [Tensorflow Data Validation(TFDV)](https://www.tensorflow.org/tfx/data_validation/get_started)은 `StatisticsGen` 및 `SchemaGen`에서 사용되며 TFDV는 또한 이러한 구성 요소의 출력에 대한 시각화를 제공합니다.

이 튜토리얼에서는, 내부적으로 TFDV를 사용하여 시각화를 표시하는 TFX의 시각화 도우미 메서드를 사용하겠습니다.

#### StatisticsGen의 출력 검사하기


In [ ]:
# docs-infra: no-execute
visualize_artifacts(stats_artifacts)

<!-- <img class="tfo-display-only-on-site"
src="images/penguin_tfdv/penguin_tfdv_statistics.png"/> -->

입력 데이터에 대한 여러 통계를 볼 수 있습니다. 이러한 통계는 `SchemaGen`로 제공되어 데이터 초기 스키마를 자동으로 구성합니다.


#### SchemaGen의 출력 검사하기


In [ ]:
visualize_artifacts(schema_artifacts)

이 스키마는 StatisticsGen의 출력에서 자동으로 추론됩니다. 4개의 FLOAT 특성과 1개의 INT 특성을 볼 수 있어야 합니다.

### 추후 사용을 위해 스키마 내보내기

생성된 스키마를 검토하고 정제해야 합니다. 이 검토된 스키마는 ML 모델 훈련을 위해 후속 파이프라인에서 사용되도록 지속되어야 합니다. 즉, 실제 활용 사례를 위해 버전 제어 시스템에 스키마 파일을 추가할 수 있습니다. 이 튜토리얼에서는, 단순화를 위해 스키마를 미리 정의된 파일 시스템 경로에 복사하기만 하면 됩니다.


In [ ]:
import shutil

_schema_filename = 'schema.pbtxt'
SCHEMA_PATH = 'schema'

os.makedirs(SCHEMA_PATH, exist_ok=True)
_generated_path = os.path.join(schema_artifacts[0].uri, _schema_filename)

# Copy the 'schema.pbtxt' file from the artifact uri to a predefined path.
shutil.copy(_generated_path, SCHEMA_PATH)

스키마 파일은 [Protocol Buffer 텍스트 형식](https://googleapis.dev/python/protobuf/latest/google/protobuf/text_format.html)과 [TensorFlow Metadata Schema proto](https://github.com/tensorflow/metadata/blob/master/tensorflow_metadata/proto/v0/schema.proto)의 인스턴스를 사용합니다.

In [ ]:
print(f'Schema at {SCHEMA_PATH}-----')
!cat {SCHEMA_PATH}/*

스키마 정의를 검토하고 필요한 대로 편집해야 합니다. 이 튜토리얼에서는, 변경되지 않은 생성된 스키마를 사용하겠습니다.


## 입력 예제 검증 및 ML 모델 훈련

[단순 TFX 파이프라인 튜토리얼](https://www.tensorflow.org/tfx/tutorials/tfx/penguin_simple)에서 생성한 파이프라인으로 돌아가 ML 모델을 훈련하고 모델 훈련 코드를 작성하기 위해 생성된 스키마를 사용하겠습니다.

스키마와 관련된 입력되는 데이터세트의 이상 및 누락된 값을 찾는 a0}ExampleValidator 구성 요소도 추가하겠습니다.


### 모델 훈련 코드 작성하기

[단순 TFX 파이프라인 튜토리얼](https://www.tensorflow.org/tfx/tutorials/tfx/penguin_simple)에서 작성했던 모델 코드를 작성해야 합니다.

모델 자체는 이전의 튜토리얼과 동일하지만 이번에는 특성을 수동으로 지정하는 대신 이전의 파이프라인에서 생성된 스키마를 사용하겠습니다. 코드 대부분은 변경되지 않았습니다. 유일한 차이점은 이 파일의 이름과 특성의 유형을 지정하지 않아도 된다는 것입니다. 대신 이를 *schema* 파일에서 읽습니다.

In [ ]:
_trainer_module_file = 'penguin_trainer.py'

In [ ]:
%%writefile {_trainer_module_file}

from typing import List
from absl import logging
import tensorflow as tf
from tensorflow import keras
from tensorflow_transform.tf_metadata import schema_utils

from tfx import v1 as tfx
from tfx_bsl.public import tfxio
from tensorflow_metadata.proto.v0 import schema_pb2

# We don't need to specify _FEATURE_KEYS and _FEATURE_SPEC any more.
# Those information can be read from the given schema file.

_LABEL_KEY = 'species'

_TRAIN_BATCH_SIZE = 20
_EVAL_BATCH_SIZE = 10

def _input_fn(file_pattern: List[str],
              data_accessor: tfx.components.DataAccessor,
              schema: schema_pb2.Schema,
              batch_size: int = 200) -> tf.data.Dataset:
  """Generates features and label for training.

  Args:
    file_pattern: List of paths or patterns of input tfrecord files.
    data_accessor: DataAccessor for converting input to RecordBatch.
    schema: schema of the input data.
    batch_size: representing the number of consecutive elements of returned
      dataset to combine in a single batch

  Returns:
    A dataset that contains (features, indices) tuple where features is a
      dictionary of Tensors, and indices is a single Tensor of label indices.
  """
  return data_accessor.tf_dataset_factory(
      file_pattern,
      tfxio.TensorFlowDatasetOptions(
          batch_size=batch_size, label_key=_LABEL_KEY),
      schema=schema).repeat()


def _build_keras_model(schema: schema_pb2.Schema) -> tf.keras.Model:
  """Creates a DNN Keras model for classifying penguin data.

  Returns:
    A Keras Model.
  """
  # The model below is built with Functional API, please refer to
  # https://www.tensorflow.org/guide/keras/overview for all API options.

  # ++ Changed code: Uses all features in the schema except the label.
  feature_keys = [f.name for f in schema.feature if f.name != _LABEL_KEY]
  inputs = [keras.layers.Input(shape=(1,), name=f) for f in feature_keys]
  # ++ End of the changed code.

  d = keras.layers.concatenate(inputs)
  for _ in range(2):
    d = keras.layers.Dense(8, activation='relu')(d)
  outputs = keras.layers.Dense(3)(d)

  model = keras.Model(inputs=inputs, outputs=outputs)
  model.compile(
      optimizer=keras.optimizers.Adam(1e-2),
      loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
      metrics=[keras.metrics.SparseCategoricalAccuracy()])

  model.summary(print_fn=logging.info)
  return model


# TFX Trainer will call this function.
def run_fn(fn_args: tfx.components.FnArgs):
  """Train the model based on given args.

  Args:
    fn_args: Holds args used to train the model as name/value pairs.
  """

  # ++ Changed code: Reads in schema file passed to the Trainer component.
  schema = tfx.utils.parse_pbtxt_file(fn_args.schema_path, schema_pb2.Schema())
  # ++ End of the changed code.

  train_dataset = _input_fn(
      fn_args.train_files,
      fn_args.data_accessor,
      schema,
      batch_size=_TRAIN_BATCH_SIZE)
  eval_dataset = _input_fn(
      fn_args.eval_files,
      fn_args.data_accessor,
      schema,
      batch_size=_EVAL_BATCH_SIZE)

  model = _build_keras_model(schema)
  model.fit(
      train_dataset,
      steps_per_epoch=fn_args.train_steps,
      validation_data=eval_dataset,
      validation_steps=fn_args.eval_steps)

  # The result of the training should be saved in `fn_args.serving_model_dir`
  # directory.
  model.save(fn_args.serving_model_dir, save_format='tf')

이제 모델 훈련을 위한 TFX 파이프라인 구축에 필요한 모든 준비 단계를 완료했습니다.

### 파이프라인 정의 작성하기

두 가지 새로운 구성 요소인 `Importer` 및 `ExampleValidator`를 추가하겠습니다. Importer는 외부 파일을 TFX 파이프라인으로 가져옵니다. 이 경우, 이것은 스키마 정의를 포함하는 파일입니다. ExampleValidator는 입력 데이터를 검사하고 모든 입력 데이터가 제공된 데이터 스키마와 일치하는지 검증할 것입니다.


In [ ]:
def _create_pipeline(pipeline_name: str, pipeline_root: str, data_root: str,
                     schema_path: str, module_file: str, serving_model_dir: str,
                     metadata_path: str) -> tfx.dsl.Pipeline:
  """Creates a pipeline using predefined schema with TFX."""
  # Brings data into the pipeline.
  example_gen = tfx.components.CsvExampleGen(input_base=data_root)

  # Computes statistics over data for visualization and example validation.
  statistics_gen = tfx.components.StatisticsGen(
      examples=example_gen.outputs['examples'])

  # NEW: Import the schema.
  schema_importer = tfx.dsl.Importer(
      source_uri=schema_path,
      artifact_type=tfx.types.standard_artifacts.Schema).with_id(
          'schema_importer')

  # NEW: Performs anomaly detection based on statistics and data schema.
  example_validator = tfx.components.ExampleValidator(
      statistics=statistics_gen.outputs['statistics'],
      schema=schema_importer.outputs['result'])

  # Uses user-provided Python function that trains a model.
  trainer = tfx.components.Trainer(
      module_file=module_file,
      examples=example_gen.outputs['examples'],
      schema=schema_importer.outputs['result'],  # Pass the imported schema.
      train_args=tfx.proto.TrainArgs(num_steps=100),
      eval_args=tfx.proto.EvalArgs(num_steps=5))

  # Pushes the model to a filesystem destination.
  pusher = tfx.components.Pusher(
      model=trainer.outputs['model'],
      push_destination=tfx.proto.PushDestination(
          filesystem=tfx.proto.PushDestination.Filesystem(
              base_directory=serving_model_dir)))

  components = [
      example_gen,

      # NEW: Following three components were added to the pipeline.
      statistics_gen,
      schema_importer,
      example_validator,

      trainer,
      pusher,
  ]

  return tfx.dsl.Pipeline(
      pipeline_name=pipeline_name,
      pipeline_root=pipeline_root,
      metadata_connection_config=tfx.orchestration.metadata
      .sqlite_metadata_connection_config(metadata_path),
      components=components)

### 파이프라인 실행


In [ ]:
tfx.orchestration.LocalDagRunner().run(
  _create_pipeline(
      pipeline_name=PIPELINE_NAME,
      pipeline_root=PIPELINE_ROOT,
      data_root=DATA_ROOT,
      schema_path=SCHEMA_PATH,
      module_file=_trainer_module_file,
      serving_model_dir=SERVING_MODEL_DIR,
      metadata_path=METADATA_PATH))

파이프라인이 성공적으로 완료되면 "INFO:absl:Component Pusher is finished." 메시지가 표시됩니다.

### 파이프라인의 출력 검사

펭귄에 대한 분류 모델을 훈련했고, ExampleValidator 구성 요소의 입력 예제도 검증했습니다. 이전 파이프라인에서 분석했던 대로 ExampleValidator의 출력을 분석할 수 있습니다.

In [ ]:
metadata_connection_config = tfx.orchestration.metadata.sqlite_metadata_connection_config(
    METADATA_PATH)

with Metadata(metadata_connection_config) as metadata_handler:
  ev_output = get_latest_artifacts(metadata_handler, PIPELINE_NAME,
                                   'ExampleValidator')
  anomalies_artifacts = ev_output[standard_component_specs.ANOMALIES_KEY]

ExampleValidator의 ExampleAnomalies 역시 시각화할 수 있습니다.

In [ ]:
visualize_artifacts(anomalies_artifacts)

예제의 각 분할에서 "No anomalies found"가 표시되어야 합니다. 이 파이프라인의 스키마 생성에 사용된 동일한 데이터를 사용했기 때문에, 예상되는 이상 사항은 여기 존재하지 않습니다. 이 파이프라인을 새로운 입력되는 데이터로 반복하여 실행하는 경우, ExampleValidator는 새로운 데이터와 기존 스키마 간의 불일치를 찾을 수 있어야 합니다.

이상이 발견되면 데이터를 검토하여 예제가 가정을 따르지 않는지 검토할 수 있습니다. StatisticsGen과 같은 기타 구성 요소의 출력이 유용할 수 있습니다. 하지만 모든 발견된 이상은 다음 파이프라인 실행을 차단하지 않습니다.

## 다음 단계

https://www.tensorflow.org/tfx/tutorials에서 더 많은 리소스를 확인할 수 있습니다.

TFX의 다양한 개념에 대해 자세히 알아보려면 [TFX 파이프라인 이해하기](https://www.tensorflow.org/tfx/guide/understanding_tfx_pipelines)를 참조하세요.
